## Functions

In [2]:
import os
from pathlib import Path

from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy_utils import database_exists, create_database
import pandas as pd

In [3]:
def get_credentials():
    path = Path.cwd() / 'config' / '.ENV'
    load_dotenv(path)
    user = os.getenv('POSTGRESUSER')
    password = os.getenv('PASSWORD')
    host = os.getenv('HOST')
    port = os.getenv('PORT')
    dbname = os.getenv('DBNAME')
    credentials = {
        'user':user,
        'password':password,
        'host':host,
        'port':port,
        'dbname':dbname,
    }
    return credentials


def get_engine(user, password, host, port, dbname):
    url = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'
    if not database_exists(url):
        create_database(url)
    engine = create_engine(url, pool_size=50, echo=False)
    return engine

def get_postgres_engine():
    keys = get_credentials()
    engine = get_engine(keys['user'],keys['password'], keys['host'],keys['port'],keys['dbname'])
    return engine

In [4]:
from sqlalchemy import Column, Integer, MetaData, String, Table, create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

from sqlalchemy.orm import sessionmaker

In [5]:
# Declaração de uma tabela
engine = get_postgres_engine()
Base = declarative_base()

class User(Base):
    __tablename__ = 'Users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)

Base.metadata.create_all(engine)

## Rollback

In [22]:
session.rollback() # type: ignore

# **CRUD**

## Create

In [13]:
Session = sessionmaker(bind=engine)
session = Session()

In [14]:
user = User(name='Joel', age=20)
user_2 = User(name='Rool', age=30)
user_3 = User(name='Melo', age=40)
user_4 = User(name='Junior', age=50)

session.add(user)
session.add_all([user_2, user_3, user_4])
session.commit()

## Read

In [15]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(engine)
session = Session()

users = session.query(User).all()
user = users[0]

print(user.id)
print(user.name)
print(user.age)

133
Melo
58


In [16]:
users = session.query(User).all()
for user in users:
    print(f'User id: {user.id}, User name: {user.name}, User age: {user.age}')

User id: 133, User name: Melo, User age: 58
User id: 134, User name: Melo, User age: 26
User id: 135, User name: Rool, User age: 25
User id: 136, User name: Arthur, User age: 58
User id: 137, User name: Arthur, User age: 72
User id: 138, User name: Melo, User age: 25
User id: 139, User name: Ramos, User age: 26
User id: 140, User name: Melo, User age: 69
User id: 141, User name: Melo, User age: 46
User id: 142, User name: Rool, User age: 69
User id: 143, User name: Anna, User age: 36
User id: 144, User name: Rool, User age: 26
User id: 145, User name: Ramos, User age: 25
User id: 146, User name: Ramos, User age: 72
User id: 147, User name: Arthur, User age: 36
User id: 148, User name: Arthur, User age: 67
User id: 149, User name: Anna, User age: 58
User id: 150, User name: Rool, User age: 26
User id: 151, User name: Victoria, User age: 58
User id: 152, User name: Arthur, User age: 72
User id: 153, User name: Joel, User age: 20
User id: 154, User name: Rool, User age: 30
User id: 155, U

In [17]:
user = session.query(User).filter_by(id=2).one_or_none()
print(user.id)
print(user.name)
print(user.age)

AttributeError: 'NoneType' object has no attribute 'id'

## Update

In [13]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(engine)
session = Session()

user = session.query(User).filter_by(id=2).one_or_none()

print(user.name)
user.name = "Outro nome"
print(user.name)

session.commit()

Rool
Outro nome


## Delete

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(engine)
session = Session()

user = session.query(User).filter_by(id=2).one_or_none()
session.delete(user)
session.commit()

# **Ordenando os Dados**

In [6]:
import random
from sqlalchemy.orm import sessionmaker

engine = get_postgres_engine()
Session = sessionmaker(engine)
session = Session()

### Resetando os dados da tabela

In [7]:
session.query(User).delete()
session.commit()

### Colocando dados falsos no banco  de dados

In [8]:
names = ['Arthur','Rool','Ramos','Melo','Anna','Victoria']
ages = [25,26,26,67,69,42,36,58,91,46,72]


for x in range(20):
    user = User(name=random.choice(names), age=random.choice(ages))
    session.add(user)
session.commit()

### Por idade do maior para o menor (decrecente)

In [9]:
users = session.query(User).order_by(User.age.desc()).all()

for user in users:
    print(user.id, user.name, user.age)

152 Arthur 72
137 Arthur 72
146 Ramos 72
142 Rool 69
140 Melo 69
148 Arthur 67
133 Melo 58
136 Arthur 58
149 Anna 58
151 Victoria 58
141 Melo 46
147 Arthur 36
143 Anna 36
139 Ramos 26
144 Rool 26
134 Melo 26
150 Rool 26
145 Ramos 25
135 Rool 25
138 Melo 25


### Por nome do menor para o maior (crecente)

In [10]:
users = session.query(User).order_by(User.name.asc()).all()

for user in users:
    print(user.id, user.name, user.age)

149 Anna 58
143 Anna 36
137 Arthur 72
152 Arthur 72
136 Arthur 58
147 Arthur 36
148 Arthur 67
141 Melo 46
140 Melo 69
133 Melo 58
138 Melo 25
134 Melo 26
139 Ramos 26
145 Ramos 25
146 Ramos 72
144 Rool 26
150 Rool 26
135 Rool 25
142 Rool 69
151 Victoria 58


# **Filtrando os Dados**

## *Iniciar sessão*

In [38]:
from sqlalchemy.orm import sessionmaker

engine = get_postgres_engine()
Session = sessionmaker(engine)
session = Session()

## *filter()*

In [24]:
# query all users
users_all = session.query(User).all()

# query all users with age greater than or equal to 25
users_filtered_age = session.query(User).filter(User.age >= 25).all()

users_filtered_age_name = session.query(User).filter(User.age >= 25, User.name == 'Arthur').all()

print(f'All users: {len(users_all)}')
print(f'users_filtered_age: {len(users_filtered_age)}')
print(f'users_filtered_age_name: {len(users_filtered_age_name)}')

All users: 24
users_filtered_age: 23
users_filtered_age_name: 5


## *filter_by()*

In [30]:
users = session.query(User).filter_by(age=58).all()

for user in users:
    print(f'user: {user.name}       age: {user.age}')


# !Com o filter by não como colocar condições

user: Melo       age: 58
user: Arthur       age: 58
user: Anna       age: 58
user: Victoria       age: 58


## *where()*

In [33]:
users = session.query(User).where(User.age > 30, User.name == 'Arthur').all()
for user in users:
    print(user.name, user.age)

Arthur 58
Arthur 72
Arthur 36
Arthur 67
Arthur 72


## *or_(), and_(), not_()* 

In [34]:
# Com a função or_()
from sqlalchemy import or_

users = session.query(User).where(
    or_(
        User.age > 30, User.name == 'Arthur'
    )
).all()

for user in users:
    print(user.name, user.age)

Melo 58
Arthur 58
Arthur 72
Melo 69
Melo 46
Rool 69
Anna 36
Ramos 72
Arthur 36
Arthur 67
Anna 58
Victoria 58
Arthur 72
Melo 40
Junior 50


In [35]:
from sqlalchemy import and_

users = session.query(User).where(
    and_(
        User.age > 30, User.name == 'Arthur'
    )
).all()

for user in users:
    print(user.name, user.age)

Arthur 58
Arthur 72
Arthur 36
Arthur 67
Arthur 72


In [37]:
from sqlalchemy import not_

users = session.query(User).where(
    not_(
        User.name == 'Arthur'
    )
).all()

for user in users:
    print(user.name, user.age)

Melo 58
Melo 26
Rool 25
Melo 25
Ramos 26
Melo 69
Melo 46
Rool 69
Anna 36
Rool 26
Ramos 25
Ramos 72
Anna 58
Rool 26
Victoria 58
Joel 20
Rool 30
Melo 40
Junior 50


## *Combinando todas elas*

In [41]:
users = session.query(User).where(
    or_(
        not_(User.name == 'Arthur'),
        and_(
            User.age > 35,
            User.age < 60
        )
    )
).all()

for user in users:
    print(user.name, user.age)

print()
print(f'numero de usuarios que atendem a essas condições : {len(users)}')

Melo 58
Melo 26
Rool 25
Arthur 58
Melo 25
Ramos 26
Melo 69
Melo 46
Rool 69
Anna 36
Rool 26
Ramos 25
Ramos 72
Arthur 36
Anna 58
Rool 26
Victoria 58
Joel 20
Rool 30
Melo 40
Junior 50

numero de usuarios que atendem a essas condições : 21


# **Agrupamento e Encadeamento**

## *Iniciar Sessão*

In [5]:
from sqlalchemy.orm import sessionmaker

engine = get_postgres_engine()
Session = sessionmaker(engine)
session = Session()

## *Agrupamento*

### Agrupamento simples

In [7]:
query =  session.query(User.age).group_by(User.age)
users = query.all()

print(query)
print(users)

SELECT "Users".age AS "Users_age" 
FROM "Users" GROUP BY "Users".age
[(36,), (69,), (25,), (30,), (50,), (40,), (46,), (26,), (72,), (20,), (67,), (58,)]


### Agrupamento com a função *func*

In [8]:
from sqlalchemy import func

# Por idade
query = session.query(User.age, func.count(User.id)).group_by(User.age)
users_age = query.all()

print(query)
print(users_age)
# Por nome
query = session.query(User.name, func.count(User.id)).group_by(User.name)
users_name = query.all()

print(query)
print(users_name)

SELECT "Users".age AS "Users_age", count("Users".id) AS count_1 
FROM "Users" GROUP BY "Users".age
[(36, 2), (69, 2), (25, 3), (30, 1), (50, 1), (40, 1), (46, 1), (26, 4), (72, 3), (20, 1), (67, 1), (58, 4)]
SELECT "Users".name AS "Users_name", count("Users".id) AS count_1 
FROM "Users" GROUP BY "Users".name
[('Victoria', 1), ('Ramos', 3), ('Joel', 1), ('Junior', 1), ('Anna', 2), ('Arthur', 5), ('Melo', 6), ('Rool', 5)]


## Encadeamento (Chaining)

In [42]:
from sqlalchemy import func

users_tuple = (
    session.query(User.age, func.count(User.id))
    .filter(User.age > 24)
    .order_by(User.age)
    .filter(User.age < 50)
    .group_by(User.age)
    .all()
)

for age, count in users_tuple:
    print(f'Age: {age} - {count} users')

Age: 25 - 3 users
Age: 26 - 4 users
Age: 30 - 1 users
Age: 36 - 2 users
Age: 40 - 1 users
Age: 46 - 1 users


In [19]:
only_arthur = True
group_by_age = True

users = session.query(User)

if only_arthur:
    users = users.filter(User.name == 'Arthur')

if group_by_age:
    users = users.group_by(User.age, User.id)

users = users.all()

for user in users:
    print(f'Users age: {user.age}, name: {user.name}') 

Users age: 72, name: Arthur
Users age: 67, name: Arthur
Users age: 72, name: Arthur
Users age: 36, name: Arthur
Users age: 58, name: Arthur


# **Relações**

## Iniciar Sessão

In [26]:
from sqlalchemy import (Column, ForeignKey, Integer, String)
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

engine = get_postgres_engine()
Session = sessionmaker(engine)
session = Session()

## **Um para muitos (1 <-> n)**

In [27]:
Base = declarative_base()

class BaseModel(Base):
    __abstract__ = True
    __allow_unmapped__ = True

    id = Column(Integer,primary_key=True)

class Address(BaseModel):
    __tablename__ = 'addresses'

    city = Column(String)
    state = Column(String)
    zip_code = Column(Integer)
    user_id = Column(ForeignKey('users.id'))

    def __repr__(self):
        return f'<Addresses(id={self.id}, city="{self.city}")>'

class User(BaseModel):
    __tablename__ = 'users'

    name = Column(String)
    age = Column(Integer)
    addresses = relationship(Address)

    def __repr__(self):
        return f'<Addresses(id={self.id}, city="{self.name}")>'

Base.metadata.create_all(engine)

In [29]:
# Criando Usuários
User1 = User(name='Joel Rool', age=23)
User2 = User(name='Arthur Ramos', age=15)

# Criando Endereços
address1 = Address(city='New York', state='NY', zip_code='10001')
address2 = Address(city='Los Andeles', state='CA', zip_code='90001')
address3 = Address(city='Chicago', state='IL', zip_code='60601')

# Associando endereços com usuarios
User1.addresses.extend([address1, address2])
User2.addresses.append(address3)

session.add(User1)
session.add(User2)
session.commit()

print(f'{User1.addresses = }')
print(f'{User2.addresses = }')


User1.addresses = [<Addresses(id=4, city="New York")>, <Addresses(id=5, city="Los Andeles")>]
User2.addresses = [<Addresses(id=6, city="Chicago")>]
